1. Get the Experience part text from the Image 

In [145]:
import os
from pdf2image import convert_from_path
import pytesseract

List the Resume

In [146]:
# Get the current directory
current_directory = os.getcwd()

# List directories in the current directory
resumes = ["datasets/resume/" + d for d in os.listdir(current_directory + "/datasets/resume")]
resumes = resumes[:10]
resumes

['datasets/resume/10554236.pdf',
 'datasets/resume/10674770.pdf',
 'datasets/resume/11163645.pdf',
 'datasets/resume/11759079.pdf',
 'datasets/resume/12065211.pdf',
 'datasets/resume/12202337.pdf',
 'datasets/resume/12338274.pdf',
 'datasets/resume/12442909.pdf',
 'datasets/resume/12780508.pdf',
 'datasets/resume/12802330.pdf']

## using OCR read the text 

In [147]:
# Replace 'input_file.pdf' with the path to your PDF file
resume_pages = []

for pdf_file in resumes:
    pages = convert_from_path(pdf_file)

    resume_pages.append(pages)

In [148]:
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

In [149]:
resumes_text = []

for pages in resume_pages:
    extracted_text = ""
    for page in pages:
        text = extract_text_from_image(page)
        extracted_text += text.replace("\n",' ') + "  " 
    resumes_text.append(extracted_text.strip())            

In [150]:
resumes_text

['ACCOUNTANT Summary  Financial Accountant specializing in financial planning, reporting and analysis within the Department of Defense.  Highlights ¢ Account reconciliations © Results-oriented © Accounting operations professional ¢ Financial reporting ¢ Analysis of financial systems © Critical thinking e ERP (Enterprise Resource Planning) software. e Excellent facilitator Accomplishments  Served on a tiger team which identified and resolved General Ledger postings in DEAMS totaling $360B in accounting adjustments. This allowed for the first successfill fiscal year-end close for 2012.  In collaboration with DFAS Europe, developed an automated tool that identified duplicate obligations. This tool allowed HQ USAFE to deobligate over $5M in duplicate obligations.  Experience Company Name July 2011 to November 2012 Accountant  City ,  State  Enterprise Resource Planning Office (ERO)  In this position as an Accountant assigned to the Defense Enterprise Accounting and Management System (DEAMS

# Task 1 Extract the Experience part of each resume

In [151]:
experience_title = ['EXPERIENCE ', 'Experience ', 'Work History ']

In [152]:
# extract the part of the Experience

experience_text = []

for resume_text in resumes_text:

    text = ''
    split_resume_text = resume_text.split(experience_title[0], 1)

    if len(split_resume_text) == 1:
        # try Experience
        for i in range(1,len(experience_title)):
            if len(split_resume_text) != 1:
                break
            split_resume_text = resume_text.split(experience_title[i], 1)

    # if the Experience is not found
    if len(split_resume_text) == 1 or len(split_resume_text[1].strip()) < 10:
        experience_text.append('No Experience Found')
        continue

    text = split_resume_text[1].strip()

    # end part Experince
    resume_title = ["EXTRA", "EDUCATION", "SKILLS", "LANGUAGES", "CERTIFICATIONS", "ACHIEVEMENTS", "CONTACT", "OBJECTIVE", "PERSONAL", "QUALIFICATIONS"
                    "ABOUT", "COURSE", "DETAILS", "PROJECTS", "INTERESTS"]

    for title in resume_title:
        text = text.split(title)[0]

    experience_text.append(text)



In [153]:
# check for people who don't have experience

for i in experience_text:
    print(i[:20])

Company Name July 20
Staff Accountant  Ja
Accountant  January 
Company Name June 20
Senior Accountant Ma
Company Name City , 
October 2011  to  Au
Accountant Aug 2015 
Accountant IT , 02/2
Lead Accountant Nove


## Process the text 

In [154]:
import re
from datetime import datetime

current_year = datetime.now().year
current_month = datetime.now().month

In [155]:
def equi_space_text(text):

    # Split the text into words
    words = text.split()
    
    # Join the words with a single space between each word
    spaced_text = ' '.join(words)

    return spaced_text

In [156]:
def process_text(text, resume_name):

    total_year = 0
    total_months = 0
    
    text_array = text.split("-")

    for i in range(len(text_array) - 1):

        # Savita's Resume.pdf problem solve
        # CV PUNITA SHAH solve
        start_date = re.search(r'\b\d{2}/\d{4}\b', text_array[i].strip())
        end_date = re.search(r'\b\d{2}/\d{4}\b', text_array[i + 1].strip()) 

        if start_date and end_date:
            
            start_date = start_date.group()
            end_date = end_date.group()

            if len(text_array[i].strip().split(start_date)[1]) == 0 and len(text_array[i + 1].strip().split(end_date)[0]) == 0:
            
                start_month, start_year = map(int, start_date.split('/'))
                end_month, end_year = map(int, end_date.split('/'))
                
                # Convert start and end dates into datetime objects
                start_datetime = datetime(start_year, start_month, 1)
                end_datetime = datetime(end_year, end_month, 1)
                
                # Calculate the difference in years and months
                num_years = end_year - start_year
                num_months = end_month - start_month
                
                # Adjust for negative months
                if num_months < 0:
                    num_years -= 1
                    num_months += 12
                
                total_year += num_years
                total_months += num_months

    print(f"Total year of work experience of {resume_name} is ","{:.1f}".format(total_year + total_months/12))

In [157]:
def convert_date(text):

    date_strs = re.findall(r'(?:\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) \d{4}\b|\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) \d{4} - \d{2}/\d{4}\b)', text)

    for date_str in date_strs:
    
        # # Parse the date string
        date_obj = datetime.strptime(date_str, '%b %Y')
    
        # # Format the date as "MM/YYYY"
        formatted_date = date_obj.strftime('%m/%Y')

        text = text.replace(date_str, formatted_date)
        
    return text

In [158]:
def months_mapping(text):
    # Define a mapping of full month names to their abbreviations
    month_mapping = {
        'january': 'jan',
        'february': 'feb',
        'march': 'mar',
        'april': 'apr',
        'may': 'may',
        'june': 'jun',
        'july': 'jul',
        'august': 'aug',
        'september': 'sep',
        'october': 'oct',
        'november': 'nov',
        'december': 'dec'
    }
        
    # Compile regular expression pattern for full month names
    pattern = re.compile(r'\b(' + '|'.join(month_mapping.keys()) + r')\b', re.IGNORECASE)
    
    # Replace full month names with abbreviations
    text = pattern.sub(lambda x: month_mapping[x.group().lower()], text)
    
    return text

In [159]:

for i in range(len(experience_text)):

    text = experience_text[i]
    resume_name = resumes[i].split('datasets/resume/')[1] 
    
    new_text = text.lower().strip()
    
    current_month_str = str(current_month)
        
    if current_month < 10:
        current_month_str = "0" + str(current_month)
        
    new_text = new_text.replace('present', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('current', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('till date', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('to', '-')
    pattern = r'[^a-z0-9/+\-.@]'
        
    # Find all matches
    new_text = re.sub(pattern, ' ', new_text)
    
    new_text = equi_space_text(new_text)
    
    new_text = months_mapping(new_text)
    new_text = convert_date(new_text)

    process_text(new_text, resume_name)

Total year of work experience of 10554236.pdf is  20.3
Total year of work experience of 10674770.pdf is  10.9
Total year of work experience of 11163645.pdf is  31.8
Total year of work experience of 11759079.pdf is  17.6
Total year of work experience of 12065211.pdf is  21.7
Total year of work experience of 12202337.pdf is  6.8
Total year of work experience of 12338274.pdf is  3.8
Total year of work experience of 12442909.pdf is  41.2
Total year of work experience of 12780508.pdf is  13.2
Total year of work experience of 12802330.pdf is  12.8


# Last year of Education

### extract the education part of resume

In [160]:
education_title = ['EDUCATION ', 'Education ', 'Qualification ', 'Academic ', 'ACADEMIC ', 'QUALIFICATION ', "QUALIFICATIONS ", "COURSE "]

In [164]:
# extract the part of the Education

education_text = []

for resume_text in resumes_text:

    text = ''
    split_resume_text = resume_text.split(education_title[0], 1)

    for i in range(1,len(education_title)):
        if len(split_resume_text) == 1 or (len(resume_text.split(education_title[i], 1)) > 1 and len(split_resume_text[1]) < len(resume_text.split(education_title[i], 1)[1])):
            split_resume_text = resume_text.split(education_title[i], 1)

    # if the Experience is not found
    if len(split_resume_text) == 1 or len(split_resume_text[1].strip()) < 10:
        education_text.append('No Eduaction Found')
        continue
    
    text = split_resume_text[1].strip()
    
    # end part Experince
    resume_title = ["EXTRA", "SKILLS", "LANGUAGES", "CERTIFICATIONS", "ACHIEVEMENTS", "CONTACT", "OBJECTIVE", "PERSONAL",
                    "ABOUT", "DETAILS", "PROJECTS", "INTERESTS", 'EXPERIENCE ', 'Work History ', 'Experience ']

    for title in resume_title:
        new_text = text.split(title)[0].strip()

        if len(new_text) == 0:

            split_resume_text = text.split(education_title[0], 1)
            
            for i in range(1,len(education_title)):
                if len(split_resume_text) == 1 or (len(resume_text.split(education_title[i], 1)) > 1 and len(split_resume_text[1]) < len(resume_text.split(education_title[i], 1)[1])):
                    split_resume_text = resume_text.split(education_title[i], 1)

    education_text.append(text)



In [165]:
# check for people who don't have education

for i in education_text:
    print(i[:20])

Northern Maine Commu
Bachelor of Science 
Computer Application
EMORY UNIVERSITY, Go
Bachelor of Business
Master of Business A
2006  Vidyasagar Uni
Bachelor of Science 
Bachelor of Science 
BS : Accounting Busi


## Process the text

In [166]:

for i in range(len(education_text)):

    text = education_text[i]
    resume_name = resumes[i].split('datasets/resume/')[1] 
    
    new_text = text.lower().strip()
    
    current_month_str = str(current_month)
        
    if current_month < 10:
        current_month_str = "0" + str(current_month)
        
    new_text = new_text.replace('present', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('current', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('till date', current_month_str + "/" + str(current_year))
    
    pattern = r'[^a-z0-9/+\-.@]'
        
    # Find all matches
    new_text = re.sub(pattern, ' ', new_text)
    
    new_text = equi_space_text(new_text)

    years = re.findall(r'(?:19|20)\d{2}', new_text)

    # Convert years to integers
    years = [int(year) for year in years]

    if len(years) != 0:
        # Find the maximum year
        max_year = max(years)
        
        print(f"The Last year of Education of {resume_name} is:", max_year)

    else:
        print(f"The Last year of Education not found for resume {resume_name}")

The Last year of Education of 10554236.pdf is: 2005
The Last year of Education of 10674770.pdf is: 2024
The Last year of Education of 11163645.pdf is: 2008
The Last year of Education of 11759079.pdf is: 2007
The Last year of Education of 12065211.pdf is: 2015
The Last year of Education of 12202337.pdf is: 2011
The Last year of Education of 12338274.pdf is: 2006
The Last year of Education not found for resume 12442909.pdf
The Last year of Education of 12780508.pdf is: 2010
The Last year of Education of 12802330.pdf is: 2005
